## Import

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# Import libraries
import pandas as pd
import numpy as np
from collections import Counter

import pydotplus 
from IPython.display import Image  

from sklearn import tree
import itertools
import math

from sklearn import preprocessing
from sklearn.metrics import roc_curve
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

## Loading Datasets

In [ ]:
# original dataset
data = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
# pre-processed
data_processed = pd.read_csv('data_post_processed_binary._virginia.csv')
# big-group
data_biggroup = pd.read_csv('telco_final_17_05.csv')

In [ ]:
data_processed.head()

# Data Understanding

**Correlation Matrix**

In [ ]:
plt.figure(figsize =(20,20))
Corr=data_biggroup[data_biggroup.columns].corr()
sns.heatmap(Corr,annot=True)

**Visualizing Attributes**

In [ ]:
to_plot_cat = ['Gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
               'InternetService', 'DSL', 'Fiber', 'PremiumServices', 'Contract', 'PaperlessBilling', 'PaymentMethod']
label_list_custom = [['Short-Term', 'Long-Term'], ['No', 'Yes'], ['Check Methods', 'Automatic Methods']]

plt.rcParams.update({'font.size': 13})

fig = plt.figure(figsize=(20,35))
_j = 0
for index, plot in enumerate(to_plot_cat):
    if index < 10:
        gender_dist = data_biggroup[[plot, 'Churn']].groupby(plot)            
        gdist_0 = list()
        gdist_1 = list()       
        _r = len(data_biggroup[plot].unique())
        if _r < 2:
            _fallback = data[data[plot] == 1]
            if len(_fallback) == 0:
                _fallback = data[data[plot] == 0]
                _index_attr = _fallback.iloc[0][plot]
                if _index_attr == 0:
                    _group_fallback = _fallback[[plot, 'Churn']].groupby('Churn')
                    gdist_0.append(len(_group_fallback.get_group(0)))
                    gdist_1.append(len(_group_fallback.get_group(1)))
                    gdist_0.append(0)
                    gdist_1.append(0)
                    # print(f'{plot} case omfg2')
                    
                    plt.subplot(5, 3, index+1)
                    plt.title(plot)
                    plt.bar(_group_fallback.get_group(0).columns, gdist_0, color='#03719c', edgecolor='white', width=0.95)
                    plt.bar(_group_fallback.get_group(0).columns, gdist_1, bottom=gdist_0, color='#ffad01', edgecolor='white', width=0.95)
                    plt.xticks((0,1),('No', 'Yes'))
                    plt.legend(['Not Churning', 'Churning'])
                    continue
                else:
                    _group_fallback = _fallback[[plot, 'Churn']].groupby('Churn')
                    gdist_0.append(0)
                    gdist_1.append(0)
                    gdist_0.append(len(_group_fallback.get_group(0)))
                    gdist_1.append(len(_group_fallback.get_group(1)))
                    # print(f'{plot} case omfg2')
                    
                    plt.subplot(5, 3, index+1)
                    plt.title(plot)
                    plt.bar(_group_fallback.get_group(0).columns, gdist_0, color='#03719c', edgecolor='white', width=0.95)
                    plt.bar(_group_fallback.get_group(0).columns, gdist_1, bottom=gdist_0, color='#ffad01', edgecolor='white', width=0.95)
                    plt.xticks((0,1),('No', 'Yes'))
                    plt.legend(['Not Churning', 'Churning'])
                    continue
            _group_fallback = _fallback[[plot, 'Churn']].groupby('Churn')
            gdist_0.append(0)
            gdist_0.append(_group_fallback.size()[0])
            gdist_1.append(0)
            gdist_1.append(_group_fallback.size()[1])
            
            # print(f'{plot} case 1')
            plt.subplot(5, 3, index+1)
            plt.title(plot)
            plt.bar(_group_fallback.get_group(0).columns, gdist_0, color='#03719c', edgecolor='white', width=0.95)
            plt.bar(_group_fallback.get_group(0).columns, gdist_1, bottom=gdist_0, color='#ffad01', edgecolor='white', width=0.95)
            plt.xticks((0,1),('No', 'Yes'))
            plt.legend(['Not Churning', 'Churning'])
            continue
            
        for i in range(len(gender_dist)):
            _temp_len = list()
            gender_df = gender_dist.get_group(i)
            churn_dist = gender_df.groupby('Churn')
            
            for j in range(len(churn_dist)):
                # print(f'Appending length {len(churn_dist.get_group(j))} for attr {plot}')
                _temp_len.append(len(churn_dist.get_group(j)))
                # print(f'{plot} case 2')
            gdist_0.append(_temp_len[0])
            gdist_1.append(0)
            if len(_temp_len) > 1:
                gdist_1.pop()
                gdist_1.append(_temp_len[1])
                # print(f'{plot} case 3')

        plt.subplot(5, 3, index+1)
        plt.bar(gender_dist.get_group(0).columns, gdist_0, color='#03719c', edgecolor='white', width=0.95)
        plt.bar(gender_dist.get_group(0).columns, gdist_1, bottom=gdist_0, color='#ffad01', edgecolor='white', width=0.95)
        plt.xticks((0,1),('No', 'Yes'))
        plt.title(plot)
        plt.legend(['Not Churning', 'Churning'])
    else:
        gender_dist = data_biggroup[[plot, 'Churn']].groupby(data_biggroup[plot])
        gdist_0 = list()
        gdist_1 = list()
        for i in range(len(gender_dist)):
            _temp_len = list()
            gender_df = gender_dist.get_group(i)
            churn_dist = gender_df.groupby('Churn')
            for j in range(len(churn_dist)):
                _temp_len.append(len(churn_dist.get_group(j)))
            gdist_0.append(_temp_len[0])
            gdist_1.append(_temp_len[1])
        plt.subplot(5, 3, index+1)
        plt.bar(gender_dist.get_group(0).columns, gdist_0, color='#03719c', edgecolor='white', width=0.95)
        plt.bar(gender_dist.get_group(0).columns, gdist_1, bottom=gdist_0, color='#ffad01', edgecolor='white', width=0.95)
        plt.xticks((0,1),label_list_custom[_j])
        plt.title(plot)
        plt.legend(['Not Churning', 'Churning'])
        _j += 1

**Barplot for premium services grouped by Churning**

In [ ]:
further_visual_cat = ['OnlineSecurityBinary' ,'OnlineBackupBinary', 'DeviceProtectionBinary', 
                      'TechSupportBinary', 'StreamingTvBinary', 'StreamingMoviesBinary']

plt.figure(figsize=(20,15))
# plt.suptitle('Deeper visualization of \'Premium Services\' by Churning', size=16)
for index, plot in enumerate(further_visual_cat):
    gender_dist = data_processed[[plot, 'Churn']].groupby(data_processed[plot])
    gdist_0 = list()
    gdist_1 = list()
    for i in range(len(gender_dist)):
        _temp_len = list()
        gender_df = gender_dist.get_group(i)
        churn_dist = gender_df.groupby('Churn')
        for j in range(len(churn_dist)):
            _temp_len.append(len(churn_dist.get_group(j)))
        gdist_0.append(_temp_len[0])
        gdist_1.append(_temp_len[1])
        

    plt.subplot(2, 3, index+1)
    plt.bar(gender_dist.get_group(0).columns, gdist_0, color='#03719c', edgecolor='white', width=0.95)
    plt.bar(gender_dist.get_group(0).columns, gdist_1, bottom=gdist_0, color='#ffad01', edgecolor='white', width=0.95)
    plt.xticks((0,1), ('No', 'Yes'))
    plt.title(plot)
    plt.legend(['Not Churning', 'Churning'])

**Barplot for premium services grouped by Connection Type**

In [ ]:
further_visual_cat = ['OnlineSecurityBinary', 'OnlineBackupBinary','DeviceProtectionBinary', 
                      'TechSupportBinary', 'StreamingTvBinary','StreamingMoviesBinary' ]

fig = plt.figure(figsize=(20,15))
# fig.suptitle('Deeper visualization of \'Premium Services\' by connection type', size=16)
for index, plot in enumerate(further_visual_cat):
    gender_dist = data_processed[[plot, 'FiberBinary']].groupby(data_processed[plot])
    gdist_0 = list()
    gdist_1 = list()
    for i in range(len(gender_dist)):
        _temp_len = list()
        gender_df = gender_dist.get_group(i)
        churn_dist = gender_df.groupby('FiberBinary')
        for j in range(len(churn_dist)):
            _temp_len.append(len(churn_dist.get_group(j)))
        gdist_0.append(_temp_len[0])
        gdist_1.append(_temp_len[1])
        

    plt.subplot(2, 3, index+1)
    plt.bar(gender_dist.get_group(0).columns, gdist_0, color='#03719c', edgecolor='white', width=0.95)
    plt.bar(gender_dist.get_group(0).columns, gdist_1, bottom=gdist_0, color='#ffad01', edgecolor='white', width=0.95)
    plt.xticks((0,1), ('No Service', 'Yes Service'))
    plt.title(plot)
    plt.legend(['DSL', 'Fiber'])

**Hisogram Tenure, MonthlyCharges and TotalCharges**

In [ ]:
data['new_churn'] = data_processed['Churn']
data.loc[data['TotalCharges'] == ' ', ['TotalCharges']] = 0
data['TotalCharges'].astype(float, inplace=True)

to_group_3 = ['tenure', 'MonthlyCharges', 'TotalCharges']
fig = plt.figure(figsize=(20, 15))
# plt.suptitle('Tenure, MonthlyCharges and TotalCharges distributions')

for index, plot in enumerate(to_group_3):
    temp_grouped = data[['MonthlyCharges', 'tenure', 'new_churn', 'TotalCharges']].groupby(data['new_churn'])

    tenure_no_churn = temp_grouped[plot].get_group(0).values.astype(float)
    tenure_churn = temp_grouped[plot].get_group(1).values.astype(float)
    
    plt.subplot(2, 2, index+1)
    plt.hist([tenure_no_churn, tenure_churn], 13, label=['Not Churning', 'Churning'], color=['#03719c', '#ffad01'])
    plt.legend(loc='upper right')
    plt.xlabel(plot)


In [ ]:
data['new_churn'] = data_processed['Churn']
data.loc[data['TotalCharges'] == ' ', ['TotalCharges']] = 0
data['TotalCharges'].astype(float, inplace=True)

to_group_4 = ['tenure', 'MonthlyCharges', 'TotalCharges','tenure', 'MonthlyCharges', 'TotalCharges']
fig = plt.figure(figsize=(25, 15))
# plt.suptitle('Tenure, MonthlyCharges and TotalCharges distributions wrt to normal and density values')

for index, plot in enumerate(to_group_4):
    temp_grouped = data[['MonthlyCharges', 'tenure', 'new_churn', 'TotalCharges']].groupby(data['new_churn'])
    if index<=2:
        tenure_no_churn = temp_grouped[plot].get_group(0).values.astype(float)
        tenure_churn = temp_grouped[plot].get_group(1).values.astype(float)

        plt.subplot(2, 3, index+1)
        plt.hist([tenure_churn, tenure_no_churn], 13, label=['Churning', 'Not Churning'], color=['#ffad01', '#03719c'], stacked=True, rwidth=0.95)
        plt.legend(loc='upper right')
        plt.xlabel(plot)
    else:
        tenure_no_churn = temp_grouped[plot].get_group(0).values.astype(float)
        tenure_churn = temp_grouped[plot].get_group(1).values.astype(float)

        plt.subplot(2, 3, index+1)
        sns.kdeplot(data[plot], color='blue', shade=True)
        plt.hist([tenure_churn, tenure_no_churn], 13, label=['Churning', 'Not Churning'], color=['#ffad01', '#03719c'], density=True, rwidth=0.95)
        plt.legend(loc='upper right')
        plt.xlabel(plot)
        

# K-Means

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
def run_kmean(X, max_k):
    sse_list = list()
    kmean_list = list()
    labels = list()
    max_k = max_k
    for k in range(2, max_k + 1):
        kmeans = KMeans(n_clusters=k, n_init=10, max_iter=100)
        res = kmeans.fit_predict(X)
        labels.append(res)
        sse = kmeans.inertia_
        sse_list.append(sse)
        kmean_list.append(kmeans)
    return sse_list, kmean_list, labels

In [ ]:
def plot_clust_distributions(data, list_to_plot):
    # plt.rcParams.update({'font.size': 13})
    to_plot_cat = list_to_plot 
    fig = plt.figure(figsize=(20,35))
    for index, plot in enumerate(to_plot_cat):
        gender_dist = data[[plot, 'Churn']].groupby(plot)            
        gdist_0 = list()
        gdist_1 = list()
        
        _r = len(data[plot].unique())
        if _r < 2:
            _fallback = data[data[plot] == 1]
            if len(_fallback) == 0:
                _fallback = data[data[plot] == 0]
                _index_attr = _fallback.iloc[0][plot]
                if _index_attr == 0:
                    _group_fallback = _fallback[[plot, 'Churn']].groupby('Churn')
                    gdist_0.append(len(_group_fallback.get_group(0)))
                    gdist_1.append(len(_group_fallback.get_group(1)))
                    gdist_0.append(0)
                    gdist_1.append(0)
                    # print(f'{plot} case omfg2')
                    
                    plt.subplot(5, 3, index+1)
                    plt.title(plot)
                    plt.bar(_group_fallback.get_group(0).columns, gdist_0, color='#03719c', edgecolor='white', width=0.95)
                    plt.bar(_group_fallback.get_group(0).columns, gdist_1, bottom=gdist_0, color='#ffad01', edgecolor='white', width=0.95)
                    plt.xticks((0,1),('No', 'Yes'))
                    plt.legend(['Not Churning', 'Churning'])
                    continue
                else:
                    _group_fallback = _fallback[[plot, 'Churn']].groupby('Churn')
                    gdist_0.append(0)
                    gdist_1.append(0)
                    gdist_0.append(len(_group_fallback.get_group(0)))
                    gdist_1.append(len(_group_fallback.get_group(1)))
                    # print(f'{plot} case omfg2')
                    
                    plt.subplot(5, 3, index+1)
                    plt.title(plot)
                    plt.bar(_group_fallback.get_group(0).columns, gdist_0, color='#03719c', edgecolor='white', width=0.95)
                    plt.bar(_group_fallback.get_group(0).columns, gdist_1, bottom=gdist_0, color='#ffad01', edgecolor='white', width=0.95)
                    plt.xticks((0,1),('No', 'Yes'))
                    plt.legend(['Not Churning', 'Churning'])
                    continue
            _group_fallback = _fallback[[plot, 'Churn']].groupby('Churn')
            gdist_0.append(0)
            gdist_0.append(_group_fallback.size()[0])
            gdist_1.append(0)
            gdist_1.append(_group_fallback.size()[1])
            
            # print(f'{plot} case 1')
            plt.subplot(5, 3, index+1)
            plt.title(plot)
            plt.bar(_group_fallback.get_group(0).columns, gdist_0, color='#03719c', edgecolor='white', width=0.95)
            plt.bar(_group_fallback.get_group(0).columns, gdist_1, bottom=gdist_0, color='#ffad01', edgecolor='white', width=0.95)
            plt.xticks((0,1),('No', 'Yes'))
            plt.legend(['Not Churning', 'Churning'])
            continue
            
        for i in range(len(gender_dist)):
            _temp_len = list()
            gender_df = gender_dist.get_group(i)
            churn_dist = gender_df.groupby('Churn')
            
            for j in range(len(churn_dist)):
                # print(f'Appending length {len(churn_dist.get_group(j))} for attr {plot}')
                _temp_len.append(len(churn_dist.get_group(j)))
                # print(f'{plot} case 2')
            gdist_0.append(_temp_len[0])
            gdist_1.append(0)
            if len(_temp_len) > 1:
                gdist_1.pop()
                gdist_1.append(_temp_len[1])
                # print(f'{plot} case 3')

        plt.subplot(5, 3, index+1)
        plt.bar(gender_dist.get_group(0).columns, gdist_0, color='#03719c', edgecolor='white', width=0.95)
        plt.bar(gender_dist.get_group(0).columns, gdist_1, bottom=gdist_0, color='#ffad01', edgecolor='white', width=0.95)
        plt.xticks((0,1),('No', 'Yes'))
        plt.title(plot)
        plt.legend(['Not Churning', 'Churning'])

In [ ]:
def print_cluster_describe_cat(cluster):
       return(cluster[['Gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 
                  'MultipleLines', 'InternetService', 'DSL', 'Fiber', 'PremiumServices',
                  'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn']].astype('str').describe())
def print_cluster_describe_cont(cluster):
        return(cluster[['Tenure_original', 'MonthlyCharges_original']].describe())

In [ ]:
def print_single_clust_dist(cluster):
    fig = plt.figure(figsize=(20,35))
    index = 0
    for attr in cluster.columns:
        if attr not in ['Tenure', 'MonthlyCharges', 'Tenure_original', 'MonthlyCharges_original', 'kmean_label']:
            print(attr)
            print(Counter(cluster[attr]))
            c = Counter(cluster[attr])
            plt.subplot(5, 4, index+1)
            plt.bar((0,1), c.values(), color= ['#03719c', '#ffad01'])
            plt.title(attr)
            plt.xticks((0,1),('No', 'Yes'))
            index += 1

In [ ]:
def plot_cont_clust(cluster, index_cluster):
    to_group_3 = ['Tenure_original', 'MonthlyCharges_original']
    fig = plt.figure(figsize=(20, 15))
    plt.suptitle('Cluster ' + str(index_cluster))
    # plt.suptitle('Tenure, MonthlyCharges and TotalCharges distributions')

    for index, plot in enumerate(to_group_3):
        temp_grouped = cluster[['MonthlyCharges_original', 'Tenure_original', 'Churn']].groupby(cluster['Churn'])

        tenure_no_churn = temp_grouped[plot].get_group(0).values.astype(float)
        tenure_churn = temp_grouped[plot].get_group(1).values.astype(float)

        plt.subplot(2, 2, index+1)
        plt.hist([tenure_no_churn, tenure_churn], 13, label=['Not Churning', 'Churning'], color=['#03719c', '#ffad01'])
        plt.legend(loc='upper right')
        plt.xlabel(plot)

In [ ]:
def get_cluster_compisition(kmeans, data):
    hist, bins = np.histogram(kmeans.labels_, bins=range(0, len(set(kmeans.labels_)) + 1))
    sizes = dict(zip(bins, hist))
    for id_cluster, size in sizes.items():
        print('Cluster %d: %d (%.2f)' % (id_cluster, size, size/len(data))+'')
        
def evaluate_cluster(k, labels, data):
    temp_df = data.copy()
    temp_df['kmeans_label'] = labels
    _matrix = np.zeros((3,2))
    temp_df2 = pd.DataFrame(_matrix)
    for k in range(k):
        _temp = temp_df[temp_df['kmeans_label'] == k]['Churn'].values
        _r = Counter(_temp)
        print(f'Cluster {k} composition wrt to target {_r}')
        print(f'ChurningRatio: {round(_r[1]/(_r[0]+_r[1])*100, 4)}')

In [ ]:
df_x_cluster = data[['tenure', 'MonthlyCharges', 'TotalCharges']]
df_x_cluster['Tenure'] = df_x_cluster['tenure']
df_x_cluster.drop(columns=['tenure'], inplace=True)

df_x_cluster[df_x_cluster['TotalCharges'] == ' '] = 0

In [ ]:
from sklearn import preprocessing

def norm_attr(data, attr):
    min_max_scaler = preprocessing.MinMaxScaler()
    _to_norm = data[attr].values
    x_scaled = min_max_scaler.fit_transform(_to_norm)
    data[attr] = x_scaled

norm_attr(df_x_cluster, ['TotalCharges'])
norm_attr(df_x_cluster, ['Tenure'])
norm_attr(df_x_cluster, ['MonthlyCharges'])

In [ ]:
df_x_cluster = df_x_cluster[['Tenure', 'MonthlyCharges', 'TotalCharges']]

In [ ]:
x_cluster = df_x_cluster.values

In [ ]:
x_cluster[0]

In [ ]:
# Search for best k
# sse_list, kmean_list, labels = run_kmean(x_cluster, 15)

silhouette_scorelist = list()
for i, km in enumerate(kmean_list):
    silhouette_scorelist.append(silhouette_score(x_cluster, km.labels_))

plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
plt.xlabel('K-value')
plt.ylabel('SSE')
plt.plot(range(2, len(sse_list) + 2), sse_list)

plt.subplot(1,2,2)
plt.xlabel('K-value')
plt.ylabel('Silhouette Score')
plt.plot(range(1, len(silhouette_scorelist)+1), silhouette_scorelist)

In [ ]:
silhouette_scorelist

In [ ]:
kmeans = KMeans(n_clusters=5, n_init=10, max_iter=100)
labels = kmeans.fit_predict(x_cluster)
silhouette_score(x_cluster, labels)

In [ ]:
get_cluster_compisition(kmeans, x_cluster)
evaluate_cluster(5, labels, data_biggroup)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

data_plotclust = data_biggroup.copy()
data_plotclust['kmean_label'] = labels
data_plotclust['Tenure_original'] = data['tenure']
data_plotclust['MonthlyCharges_original'] = data['MonthlyCharges']

groups_kmeans = data_plotclust.groupby(data_plotclust['kmean_label'])

# Plot
fig, ax = plt.subplots(figsize=(15,10))
ax = Axes3D(fig)
ax.margins(0.02) 
for name, group in groups_kmeans:
    name = 'Cluster ' +str(name)
    ax.scatter(group['Tenure_original'],  group['MonthlyCharges_original'],group['Churn'],  marker='o', s=20, label=name)
ax.legend()
ax.set_xlabel('Tenure')
ax.set_ylabel('Monthly Charges')
ax.set_zlabel('Churn')

plt.show()

In [ ]:
kmenas_cluster_0 = groups_kmeans.get_group(0) 
plot_clust_distributions(kmenas_cluster_0, ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'DSL', 
                               'Fiber', 'PremiumServices','Contract', 'PaperlessBilling', 
                               'PaymentMethod'])

In [ ]:
print_cluster_describe_cat(kmenas_cluster_0)

In [ ]:
print_cluster_describe_cont(kmenas_cluster_0)

In [ ]:
# NO PhoneService nor PremiumServices
kmenas_cluster_1 = groups_kmeans.get_group(1) 
plot_clust_distributions(kmenas_cluster_1, ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'DSL', 
                               'Fiber', 'PremiumServices', 'Contract', 'PaperlessBilling', 'PaymentMethod'])

In [ ]:
print_cluster_describe_cat(kmenas_cluster_1)

In [ ]:
print_cluster_describe_cont(kmenas_cluster_1)

In [ ]:
kmenas_cluster_2 = groups_kmeans.get_group(2) 
plot_clust_distributions(kmenas_cluster_2, ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'DSL', 
                               'Fiber', 'PremiumServices', 'Contract', 'PaperlessBilling', 
                               'PaymentMethod'])

In [ ]:
print_cluster_describe_cat(kmenas_cluster_2)

In [ ]:
print_cluster_describe_cont(kmenas_cluster_2)

In [ ]:
kmenas_cluster_3 = groups_kmeans.get_group(3) 
plot_clust_distributions(kmenas_cluster_3, ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'DSL', 
                               'Fiber','PremiumServices', 'Contract', 'PaperlessBilling', 
                               'PaymentMethod'])

In [ ]:
print_cluster_describe_cat(kmenas_cluster_3)

In [ ]:
print_cluster_describe_cont(kmenas_cluster_3)

In [ ]:
kmenas_cluster_4 = groups_kmeans.get_group(4) 
plot_clust_distributions(kmenas_cluster_4, ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'DSL', 
                               'Fiber', 'PremiumServices', 'Contract', 'PaperlessBilling', 'PaymentMethod'])

In [ ]:
print_cluster_describe_cat(kmenas_cluster_4)

In [ ]:
print_cluster_describe_cont(kmenas_cluster_4)

In [ ]:
plot_cont_clust(kmenas_cluster_0, 0)

In [ ]:
plot_cont_clust(kmenas_cluster_1, 1)

In [ ]:
plot_cont_clust(kmenas_cluster_2, 2)

In [ ]:
plot_cont_clust(kmenas_cluster_3, 3)

In [ ]:
plot_cont_clust(kmenas_cluster_4, 4)

# DBscan

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
data_postdbscan = data_plotclust.copy()

In [ ]:
# Normalizing continuous values
x_cluster_DBSCAN = df_x_cluster.values

In [ ]:
# manual search best eps could be 0.02 and 5 min sample

dbscan = DBSCAN(eps=0.052, min_samples=80, metric='euclidean')
dbscan.fit(x_cluster_DBSCAN)
dbscan_label = dbscan.fit_predict(x_cluster_DBSCAN)
Counter(dbscan_label)

In [ ]:
def run_search_dbscan(eps_list):
    noise_value = list()
    cluster_number = list()
    label_list = list()
    for eps in eps_list:
        dbscan = DBSCAN(eps=eps, min_samples=80, metric='euclidean')
        dbscan.fit(x_cluster_DBSCAN)
        dbscan_label = dbscan.fit_predict(x_cluster_DBSCAN)
        c = Counter(dbscan_label)
        noise_value.append(c[-1])
        cluster_number.append(len(c)-1)
        label_list.append(dbscan_label)
    return noise_value, cluster_number, label_list

In [ ]:
eps_list = np.linspace(0.001,0.2,100).tolist()
noise_value_dbscan, cluster_number_dbscan, label_list_dbscan = run_search_dbscan(eps_list)

In [ ]:
plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
plt.xlabel('Eps')
plt.ylabel('Noise Amount')
plt.plot(eps_list, noise_value_dbscan)

plt.subplot(1,2,2)
plt.xlabel('Eps')
plt.ylabel('Clusters')
plt.plot(eps_list, cluster_number_dbscan)

In [ ]:
eps_list[28]
cluster_number_dbscan[28]
noise_value_dbscan[28]
label_list_dbscan[28]
Counter(label_list_dbscan[28])

In [ ]:
# Chose cluster 28 - eps 0.05728282828282828
data_postdbscan['dbscan_label'] = label_list_dbscan[28]
data_postdbscan.head()

In [ ]:
groups_dbscan = data_postdbscan.groupby(data_postdbscan['dbscan_label'])

# Plot
fig, ax = plt.subplots(figsize=(20,15))
ax = Axes3D(fig)
ax.margins(0.02)
marker_list = ['o', 'v','D', '^', '<', '>', '1', '2', '3','4','s']
i = 0
for name, group in groups_dbscan:
    if i == 0:
        i += 1 # avoid plotting 'noise' points
        continue
    name = 'Cluster ' +str(name)
    ax.scatter(group['Tenure_original'], group['MonthlyCharges_original'], group['Churn'], marker=marker_list[0], s=25, label=name)
    i += 1
ax.legend()
ax.set_xlabel('Tenure')
ax.set_ylabel('Monthly Charges')
ax.set_zlabel('Churn')

plt.show()
del(i)

In [ ]:
dbscan_cluster_0 = groups_dbscan.get_group(0) 
plot_clust_distributions(dbscan_cluster_0, ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'DSL', 
                               'Fiber', 'PremiumServices','Contract', 'PaperlessBilling', 
                               'PaymentMethod'])

In [ ]:
print_cluster_describe_cat(dbscan_cluster_0)

In [ ]:
print_cluster_describe_cont(dbscan_cluster_0)

In [ ]:
Counter(dbscan_cluster_4['Churn'])

In [ ]:
dbscan_cluster_1 = groups_dbscan.get_group(1) 
plot_clust_distributions(dbscan_cluster_1, ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'DSL', 
                               'Fiber', 'PremiumServices','Contract', 'PaperlessBilling', 
                               'PaymentMethod'])

In [ ]:
print_cluster_describe_cat(dbscan_cluster_1)

In [ ]:
print_cluster_describe_cont(dbscan_cluster_1)

In [ ]:
dbscan_cluster_2 = groups_dbscan.get_group(2) 
plot_clust_distributions(dbscan_cluster_2, ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'DSL', 
                               'Fiber', 'PremiumServices','Contract', 'PaperlessBilling', 
                               'PaymentMethod'])

In [ ]:
print_cluster_describe_cat(dbscan_cluster_2)

In [ ]:
print_cluster_describe_cont(dbscan_cluster_2)

In [ ]:
dbscan_cluster_3 = groups_dbscan.get_group(3) 
plot_clust_distributions(dbscan_cluster_3, ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'DSL', 
                               'Fiber', 'PremiumServices','Contract', 'PaperlessBilling', 
                               'PaymentMethod'])

In [ ]:
print_cluster_describe_cat(dbscan_cluster_3)

In [ ]:
print_cluster_describe_cont(dbscan_cluster_3)

In [ ]:
dbscan_cluster_4 = groups_dbscan.get_group(4) 
plot_clust_distributions(dbscan_cluster_4, ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'DSL', 
                               'Fiber', 'PremiumServices','Contract', 'PaperlessBilling', 
                               'PaymentMethod'])

In [ ]:
print_cluster_describe_cat(dbscan_cluster_4)

In [ ]:
print_cluster_describe_cont(dbscan_cluster_4)

In [ ]:
# data_plotclust.to_csv('datapostclust_21_05.csv', index=False)

In [ ]:
plot_cont_clust(dbscan_cluster_0, 0)

In [ ]:
plot_cont_clust(dbscan_cluster_1, 1)

In [ ]:
plot_cont_clust(dbscan_cluster_2, 2)

In [ ]:
plot_cont_clust(dbscan_cluster_3, 3)

In [ ]:
plot_cont_clust(dbscan_cluster_4, 4)

# Hierarchical Clustering

In [ ]:
from scipy.stats import mode
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, dendrogram, complete, fcluster, cut_tree

In [ ]:
data_postclust = pd.read_csv('datapostclust_21_05.csv')

In [ ]:
def fancy_dendrogram(*args, **kwargs):
    max_d = kwargs.pop('max_d', None)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    annotate_above = kwargs.pop('annotate_above', 0)

    ddata = dendrogram(*args, **kwargs)

    if not kwargs.get('no_plot', False):
        plt.title('Hierarchical Clustering Dendrogram (truncated)')
        plt.xlabel('sample index or (cluster size)')
        plt.ylabel('distance')
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            if y > annotate_above:
                plt.plot(x, y, 'o', c=c)
                plt.annotate("%.3g" % y, (x, y), xytext=(0, -5),
                             textcoords='offset points',
                             va='top', ha='center')
        if max_d:
            plt.axhline(y=max_d, c='k')
    return ddata

In [ ]:
Z = linkage(x_cluster, 'complete')

In [ ]:
plt.figure(figsize=(25,10))
fancy_dendrogram(
    Z,
    truncate_mode='level',
    p=30,
    leaf_rotation=45.,
    leaf_font_size=0.,
    show_contracted=True,
    annotate_above=0.5,
    max_d=0.80,
)
plt.show()

In [ ]:
Z

In [ ]:
test_labels_hier = fcluster(Z, 0.8, criterion='distance')

In [ ]:
# data_plotclust['label_hier_complete'] = test_labels_hier
# data_postdbscan['label_hier_complete'] = test_labels_hier
data_postclust['label_hier_complete'] = test_labels_hier

In [ ]:
Z2 = linkage(x_cluster, 'single')

In [ ]:
plt.figure(figsize=(15,20))
fancy_dendrogram(
    Z2,
    truncate_mode='lastp',
    p=30,
    leaf_rotation=90.,
    leaf_font_size=12.,
    show_contracted=True,
    annotate_above=0.05,
    max_d=0.033,
)
plt.show()

In [ ]:
data_postclust['label_hier_complete'].unique()

# Compute Silhouette 

In [ ]:
data_imported = pd.read_csv('node_FINAL21_05_nonoise2.csv')

In [ ]:
silhouette_score(x_cluster, data_postclust['label_hier_complete'])

In [ ]:
silhouette_score(x_cluster, data_postclust['kmean_label'])

In [ ]:
silhouette_score(x_cluster, dbscan_label2)

In [ ]:
# Retrieve label dbscan - forgo to check silhouette in dbscan section
dbscan2 = DBSCAN(eps=0.05728282828282828, min_samples=80, metric='euclidean')
dbscan2.fit(x_cluster)
dbscan_label2 = dbscan2.fit_predict(x_cluster)
Counter(dbscan_label2)

In [ ]:
db_silt = data_biggroup.copy()
db_silt['dbscan_label'] = dbscan_label2
db_silt[['Tenure_or', 'MonthlyCharges_or', 'TotalCharges_or']]

In [ ]:
x_db_silt =  data[['tenure', 'MonthlyCharges', 'TotalCharges']]

In [ ]:
x_cluster_dbscan_nonoise = df_x_cluster_dbscan_sil[['MonthlyCharges', 'TotalCharges', 'Tenure']].values
x_cluster_dbscan_nonoise

In [ ]:
# Retrieve label dbscan - forgo to check silhouette in dbscan section
dbscan3 = DBSCAN(eps=0.05728282828282828, min_samples=80, metric='euclidean')
# dbscan3.fit(x_cluster_dbscan_nonoise)
dbscan_label3 = dbscan3.fit_predict(x_cluster_dbscan_nonoise)
Counter(dbscan_label3)

In [ ]:
silhouette_score(x_cluster_dbscan_nonoise, dbscan_label3)

# Association Rules

In [ ]:
import fim

In [ ]:
data_biggroup_mining = data_biggroup.copy()
data_biggroup_mining.head()

In [ ]:
data_biggroup_mining_reduced = data_biggroup_mining[['PhoneService', 'InternetService', 'DSL', 'Fiber', 'PremiumServices', 'Contract',
                                                     'PaperlessBilling', 'PaymentMethod', 'Tenure', 'MonthlyCharges', 'Churn']]
new_name_reduced = ['_phSer', '_intSer', '_DSL', '_Fiber', '_premServ', '_contr', '_pLBil', '_paMe', '_tenure', '_monCh', '_churn']
for i, col_name in enumerate(data_biggroup_mining_reduced.columns):
    data_biggroup_mining_reduced[col_name] = data_biggroup_mining_reduced[col_name].astype(str) + str(new_name_reduced[i])

In [ ]:
data_biggroup_mining_reduced.head()

In [ ]:
new_name = ['_gen', '_sen', '_par', '_dep', '_phSer', '_mulLin', '_intSer', '_DSL', '_Fiber',
            '_premServ', '_contr', '_pLBil', '_paMe', '_tenure', '_monCh', '_churn']
for i, col_name in enumerate(data_biggroup.columns):
    data_biggroup_mining[col_name] = data_biggroup_mining[col_name].astype(str) + str(new_name[i])

In [ ]:
data_biggroup_mining.head()

In [ ]:
class Decoded_rules():
    def __init__(self, encoded_rules):
        self.encoded_rules = encoded_rules
        self.tree = {}
        self._init_tree()
        self.tree_keys = None
        
    def _init_tree(self):
        set_keys = list()
        for key in self.encoded_rules:
            set_keys.append(key['target'])
        set_keys = set(set_keys)
        for key in set_keys:
            self.tree[key] = list()
        # populate the tree
        for rule in self.encoded_rules:
            tar = rule['target']
            self.tree[tar].append((rule['rule'], rule['asb_support'], round(rule['confidence'],8), round(rule['lift'],8)))
        self.tree_keys = self.tree.keys()
        
    def search_rule(self, min_conf, target):
        res = list()
        for tar in target:
            if tar not in list(self.tree.keys()):
                res.append((tar, 'No Rules Found'))
            else:
                r = list()
                sub_tree = self.tree[tar]
                for rule in sub_tree:
                    if rule[-1] > min_conf:
                        r.append(rule)
                res.append((tar,r))
        return res
            
    def get_rule(self, index):
        rule = self.encoded_rules[index]
        
        pattern_rule = ''
        for elem in rule['rule']:
            pattern_rule += '\n' + elem
        tar = rule['target']
        abs_sup = rule['asb_support']
        rlt_sup = rule['rlt_support']
        conf = rule['confidence']
        lift = rule['lift']
        
        print(f"Target: \n{tar}\n\nRule: {pattern_rule}\n\nAbs value support: {abs_sup}\nRlt value support: {rlt_sup}\nConfidence: {conf}\nLift: {lift}")
    
def decode_rules(rules):
    test = list()
    rules.sort(key=lambda rule: rule[5], reverse=True)
    print(f'Found {len(rules)} rules\n')
    for rule in rules:
        decoded = {'target':rule[0], 'rule':rule[1], 'asb_support':rule[2], 'rlt_support':rule[3], 'confidence':rule[4], 'lift':rule[5]}
        test.append(decoded)
    
    dec_rules = Decoded_rules(test)
    return dec_rules, test

In [ ]:
def print_rule(temp):
    target = temp[0]
    final = 'Target: ' + str(target) + '\n\n'
    _temp = temp
    for elem in temp[1]:
        temp = elem.split('_')
        
        if temp[1] == 'gen':
            inv_map_val = {0:'Female Customer', 1:'Male Customer'}
            final += inv_map_val[int(temp[0])] + '\n'            
            
        elif temp[1] == 'sen':
            inv_map_val = {0:'No Internet Service', 1:'Yes Internet Service'}
            final += inv_map_val[int(temp[0])] + '\n'
        
        elif temp[1] == 'par':
            inv_map_val = {0:'No Partner', 1:'Yes Partner'}
            final += inv_map_val[int(temp[0])] + '\n'
        
        elif temp[1] == 'dep':
            inv_map_val = {0:'No Dependents', 1:'Yes Dependents'}
            final += inv_map_val[int(temp[0])] + '\n'
        
        elif temp[1] == 'phSer':
            inv_map_val = {0:'No', 1:'Yes'}
            final += inv_map_val[int(temp[0])] + ' Phone Service\n'
            
        elif temp[1] == 'mulLin':
            inv_map_val = {0:'No Multi Lines', 1:'Yes Multi Lines'}
            final += inv_map_val[int(temp[0])] + '\n'
        
        elif temp[1] == 'intSer':
            inv_map_val = {0:'No Internet Service', 1:'Yes Internet Service'}
            final += inv_map_val[int(temp[0])] + '\n'
            
        elif temp[1] == 'DSL':
            inv_map_val = {0:'No DSL', 1:'Yes DSL'}
            final += inv_map_val[int(temp[0])] + '\n'
            
        elif temp[1] == 'Fiber':
            inv_map_val = {0:'No Fiber', 1:'Yes Fiber'}
            final += inv_map_val[int(temp[0])] + '\n'
        
        elif temp[1] == 'premServ':
            inv_map_val = {0:'No premium Services', 1:'Yes Premium Services'}
            final += inv_map_val[int(temp[0])] + '\n'
       
        elif temp[1] == 'pLBil':
            inv_map_val = {0:'No Paperless Billing', 1:'Yes Paperless Billing'}
            final += inv_map_val[int(temp[0])] + '\n'
        
        elif temp[1] == 'contr':
            inv_map_val = {0:'Month-to-month', 1:'Long-term Contract'}
            final += inv_map_val[int(temp[0])] + ' Contract\n'
            
        elif temp[1] == 'paMe':
            inv_map_val = {0:'Check', 1:'Automatic'}
        
        elif temp[1] == 'tenure':
            final += temp[0] + ' (over 13 categories) Tenure range\n'
        
        elif temp[1] == 'monCh':
            final += temp[0] + ' (over 13 categories) MonthlyCharges range\n'
            
        elif temp[1] == 'churn':
            inv_map_val = {0:'No Churn', 1:'Yes Churn'}
            final += inv_map_val[int(temp[0])] + '\n'
        else:
            final += temp[1] + ' TODO!\n'
            
    lift_value = _temp[-1]
    conf_value = _temp[-2]
    final += '\nConfidence score: ' + str(conf_value) + '\nLift score: ' + str(lift_value)
    print(final)

In [ ]:
def print_rule_from_tree(temp):
    target = temp[0]
    final = 'Target: ' + str(target) + '\n\n'
    _temp = temp
    for elem in temp[1][0]:
        temp = elem.split('_')
        if temp[1] == 'gen':
            inv_map_val = {0:'Female Customer', 1:'Male Customer'}
            final += inv_map_val[int(temp[0])] + '\n'            
            
        elif temp[1] == 'sen':
            inv_map_val = {0:'No Internet Service', 1:'Yes Internet Service'}
            final += inv_map_val[int(temp[0])] + '\n'
        
        elif temp[1] == 'par':
            inv_map_val = {0:'No Partner', 1:'Yes Partner'}
            final += inv_map_val[int(temp[0])] + '\n'
                
        elif temp[1] == 'dep':
            inv_map_val = {0:'No Dependents', 1:'Yes Dependents'}
            final += inv_map_val[int(temp[0])] + '\n'
            
        elif temp[1] == 'phSer':
            inv_map_val = {0:'No', 1:'Yes'}
            final += inv_map_val[int(temp[0])] + ' Phone Service\n'
            
        elif temp[1] == 'mulLin':
            inv_map_val = {0:'No Multi Lines', 1:'Yes Multi Lines'}
            final += inv_map_val[int(temp[0])] + '\n'
        
        elif temp[1] == 'intSer':
            inv_map_val = {0:'No Internet Service', 1:'Yes Internet Service'}
            final += inv_map_val[int(temp[0])] + '\n'
            
        elif temp[1] == 'DSL':
            inv_map_val = {0:'No DSL', 1:'Yes DSL'}
            final += inv_map_val[int(temp[0])] + '\n'
            
        elif temp[1] == 'Fiber':
            inv_map_val = {0:'No Fiber', 1:'Yes Fiber'}
            final += inv_map_val[int(temp[0])] + '\n'
        
        elif temp[1] == 'premServ':
            inv_map_val = {0:'No premium Services', 1:'Yes Premium Services'}
            final += inv_map_val[int(temp[0])] + '\n'
       
        elif temp[1] == 'pLBil':
            inv_map_val = {0:'No Paperless Billing', 1:'Yes Paperless Billing'}
            final += inv_map_val[int(temp[0])] + '\n'
        
        elif temp[1] == 'contr':
            inv_map_val = {0:'Month-to-month', 1:'Long-term Contract'}
            final += inv_map_val[int(temp[0])] + ' Contract\n'
            
        elif temp[1] == 'paMe':
            inv_map_val = {0:'Check', 1:'Automatic'}
        
        elif temp[1] == 'tenure':
            final += temp[0] + ' (over 13 categories) Tenure range\n'
        
        elif temp[1] == 'monCh':
            final += temp[0] + ' (over 13 categories) MonthlyCharges range\n'
            
        elif temp[1] == 'churn':
            inv_map_val = {0:'No Churn', 1:'Yes Churn'}
            final += inv_map_val[int(temp[0])] + '\n'
        else:
            final += temp[1] + ' TODO!\n'
            
    lift_value = _temp[1][-1]
    conf_value = _temp[1][-2]
    final += '\nConfidence score: ' + str(conf_value) + '\nLift score: ' + str(lift_value)
    print(final)

In [ ]:
def explore_tree_todepth(decoded_rules, depth):
    for target in decoded_rules.tree.keys():
        print('#'*50)
        print(f'TARGET: {target}')
        if len(decoded_rules.tree[target]) == 0:
            print(f'Couldn\'t found any rule for {target}')
        else:
            if len(decoded_rules.tree[target]) > depth:
                for i in range(depth):
                    print(f'_____________________________________________________\n\nRule {i} of {depth}:')
                    print_rule_from_tree((target, decoded_rules.tree[target][i]))
                    print('\n')
            else:
                max_len = len(decoded_rules.tree[target])
                for i in range(max_len):
                    print(f'_____________________________________________________\n\nRule {i} of {max_len}:')
                    print_rule_from_tree((target, decoded_rules.tree[target][i]))
                    print('\n')

In [ ]:
def rule_based_classifier(data, target, rules, verbose=False):
    dict_affix_column = {'tenure':'Tenure','totCh':'TotalCharges','monCh':'MonthlyCharges','webSer':'InternetService',
                         'teSup': 'TechSupport','onSec':'OnlineSecurity','onBk':'OnlineBackup','dvPro':'DeviceProtection',
                         'strTv':'StreamingTv','strMv':'StreamingMovies','contr':'Contract','churn':'Churn',
                         'pLBil':'PaperlessBilling','phSer':'PhoneService','gen':'Gender','sen':'SeniorCitizen',
                         'par':'Partner','dep':'Dependents','mulLin':'MultipleLines', 
                         'paMe':'PaymentMethod','strServ':'StreamingService', 'intSer':'InternetService',
                        'DSL':'DSL', 'Fiber':'Fiber', 'premServ':'PremiumServices'}
    r = list()
    
    churning=True

    for i in range(len(data)):
        if verbose:
            print(f'For sample at index {i}')
        r_temp = list()
        for rule in rules:
            # print(rule)
            _temp_rule_for_sample = list()
            for flag in rule[0]:
                # print(flag)
                _flag = flag.split('_')
                if churning:
                    _temp_rule_for_sample.append(data.iloc[i][dict_affix_column[_flag[1]]] == flag)
                else:
                    _temp_rule_for_sample.append(data.iloc[i][dict_affix_column[_flag[1]]] != flag)  
            r_temp.append((all(_temp_rule_for_sample)))
        if verbose:
            print(f'\nList of bool for sample: {r_temp}')
            print(f'Final bool for sample at index {i}: {all(r_temp)}')
            print(f'Appendig as final result for this sample: {any(r_temp)}\n')
            print('#'*100)
        r.append(any(r_temp))
    return r

In [ ]:
def evaluate_rule_based(data, target_val, pred, is_target_rule_0_churn):
    temp_acc = 0
    tp = 1
    tn = 1
    fp = 1
    fn = 1
    
    for i, result in enumerate(pred):
        temp = (data['Churn'].iloc[i].split('_')[0])
        
        if is_target_rule_0_churn:
            result = not(result)
        
        if result == int(data[target_val].iloc[i].split('_')[0]):
            temp_acc += 1
            if result == 1:
                tp += 1
            else:
                tn += 1
        else:
            if result == 1:
                fp += 1
            else:
                fn += 1

    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    f1 = 2 * ((precision*recall)/(precision+recall))
    res = [accuracy, precision, recall, f1]
    return res

**Association Rules**

In [ ]:
basket = data_biggroup_mining.values
rules = fim.apriori(basket, supp=20, zmin=2, target='r', conf=80, report='ascl')

In [ ]:
decoded_rules, ordered_rules = decode_rules(rules)
explore_tree_todepth(decoded_rules, 1)

In [ ]:
for rule in ordered_rules:
    if len(rule['rule']) > 6:
        print(rule['target'])
        print(rule['rule'])
        print(rule['lift'])
        print('#'*50)
        print('\n')

In [ ]:
for rule in ordered_rules:
    if rule['target'] == '0_churn' and rule['lift'] > 1.3:
        print(rule['target'])
        print(rule['rule'])
        print(rule['lift'])
        print(rule['asb_support'])
        print('#'*50)
        print('\n')

In [ ]:
basket_reduced = data_biggroup_mining_reduced.values

In [ ]:
rules_reduced = fim.apriori(basket_reduced, supp=20, zmin=2, target='r', conf=80, report='ascl')

In [ ]:
decoded_rules_reduced, ordered_rules_reduced = decode_rules(rules_reduced)
# explore_tree_todepth(decoded_rules_reduced, 1)

In [ ]:
to_avoid = list()
current_target = ' '
for rule in ordered_rules_reduced:
    if rule['target'] not in to_avoid:
        print(rule['target'])
        to_avoid.append(rule['target'])
        print(rule['rule'])
        print(rule['rlt_support'])
        print(rule['lift'])
        print('#'*50)
        print('\n')

**Frequent Pattern**

In [ ]:
frequent_patterns = fim.apriori(basket_reduced, supp=20, zmin=3, target = 's')

In [ ]:
len(frequent_patterns)

In [ ]:
frequent_patterns.sort(key=lambda pattern: pattern[1], reverse=True)
frequent_patterns
frequent_patterns[:5]

In [ ]:
closed_patterns = fim.apriori(basket_reduced, supp=20, zmin=3, target = 'c')
closed_patterns.sort(key=lambda pattern: pattern[1], reverse=True)
closed_patterns[:5]

In [ ]:
maximal_patterns = fim.apriori(basket_reduced, supp=20, zmin=3, target = 'm')
maximal_patterns.sort(key=lambda pattern: pattern[1], reverse=True)
maximal_patterns[:5]

In [ ]:
df_x_cluster.to_csv('continuous_standar.csv', index=False)

**Looking fro 1_churn**

In [ ]:
rules_reduced_lf1 = fim.apriori(basket_reduced, supp=1, zmin=2, target='r', conf=80, report='ascl')
decoded_rules_reduced_f1, ordered_rules_reduced_f1 = decode_rules(rules_reduced_lf1)

In [ ]:
to_avoid = list()
current_target = ' '
for rule in ordered_rules_reduced_f1:
    if rule['target'] not in to_avoid:
        print(rule['target'])
        to_avoid.append(rule['target'])
        print(rule['rule'])
        print(rule['rlt_support'])
        print(rule['lift'])
        print('#'*50)
        print('\n')

**Rule-based classifier**

In [ ]:
#Creating function for Confusion Matrix , Precsion, Recall and F1 Score
def plot_confusion_matrix(classifier, y_test, y_pred_test):
    cm = confusion_matrix(y_test, y_pred_test)
    
    plt.clf()
    plt.imshow(cm, interpolation='nearest', cmap='RdBu')
    classNames = ['Churn-No','Churn-Yes']
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.colorbar()
    tick_marks = np.arange(len(classNames))
    plt.xticks(tick_marks, classNames, rotation=45)
    plt.yticks(tick_marks, classNames)
    s = [['TN','FP'], ['FN', 'TP']]
    
    for i in range(2):
        for j in range(2):
            plt.text(j,i, str(s[i][j])+" = "+str(cm[i][j]), 
                     horizontalalignment='center', color='White')
    
    plt.show()
        
    tn, fp, fn, tp = cm.ravel()

    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    F1 = 2*recall*precision/(recall+precision)
    accuracy = (tp + tn)/(tp+tn+fn+fp)

    print(f'Accuracy: {round(accuracy,5)*100}%')
    print('Recall: {0:0.3f}%'.format(recall*100),'\nPrecision: {0:0.3f}%'.format(precision*100))
    print('F1: {0:0.3f}%'.format(F1*100))
    return;

In [ ]:
# extracting rules from 80% dataset [:5634]
basket_tr = data_biggroup_mining[:5634].values
rules_tr = fim.apriori(basket_tr, supp=20, zmin=2, target='r', conf=80, report='ascl')
decoded_rules_reduced_tr, ordered_rules_reduced_tr = decode_rules(rules_tr)

In [ ]:
not_churn = decoded_rules_reduced.search_rule(1, ['0_churn'])

In [ ]:
r = not_churn[0][1][:5]

In [ ]:
r[0]

In [ ]:
rule_cl_pred = rule_based_classifier(data_biggroup_mining[:5634], 'Churn', r, False)

In [ ]:
# EVALUATE ON TRAINING
evaluate_rule_based(data_biggroup_mining[:5634], 'Churn', rule_cl_pred, True)

In [ ]:
# EVALIATE ON TEST
rule_cl_pred_test =rule_based_classifier(data_biggroup_mining[5634:], 'Churn', r, False)
evaluate_rule_based(data_biggroup_mining[5634:], 'Churn', rule_cl_pred_test, True)

**Plotting some plot for pattern mining**

In [ ]:
y_to_plot = list()
rules_list = list()
for i in range(20,40):
    rules_reduced = fim.apriori(basket_reduced, supp=i, zmin=5, target='r', conf=80, report='ascl')
    rules_list.append(rules_reduced)
    y_to_plot.append(len(rules_reduced))

In [ ]:
fig = plt.figure(figsize=(20,20))
plt.plot(range(20,40), y_to_plot)
plt.grid(True)
plt.xlabel('min_sup')
plt.ylabel('number of rules')

In [ ]:
decoded_rules_reduced, ordered_rules_reduced = decode_rules(rules_list[19])

In [ ]:
ordered_rules_reduced